In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn.functional as F
import torch

In [2]:
df = pd.read_csv("datasets/breast-cancer-wisconsin.csv",sep=',')
df = shuffle(df)
train_end = 500

train_df = df[:train_end]
test_df = df[train_end:]

In [3]:
def getDS(df):
    df1 = df.iloc[:,1:10]
    df2 = df.iloc[:,10]
    y = np.array(df2)
    y[y==4]=1 # malignant
    y[y==2]=0 # benign
    X = np.array(df1)
    X = X.astype(float)
    return X,y

In [4]:
train = getDS(train_df)
test = getDS(test_df)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 50)
        self.fc2 = nn.Linear(50, 2)
        self.scores = nn.Softmax()

    def forward(self, x):
        x = F.tanh(self.fc1(x))
#         print x.grad_fn
        x = self.fc2(x)
        x = self.scores(x)
        return x

In [6]:

# pla = Net()
# pla = BinaryNetwork(9,50,2)
# for i, data in enumerate(train_dataloader, 0):

#     # get the inputs
#     inputs, labels = data['X'], data['classes']
# # wrap them in Variable
# inputs, labels = Variable(inputs.float()), Variable(labels)
# optimizer = optim.SGD(pla.parameters(), lr=0.001, momentum=0.9)

# # zero the parameter gradients
# optimizer.zero_grad()
# pla.forward(inputs)

In [7]:
class CancerDataset(Dataset):
    """CancerDataset."""

    def __init__(self, X, y):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.X = X
        self.y = y

    def __len__(self):
        return (self.X.shape[0])

    def __getitem__(self, idx):
        sample = {'X': self.X[idx], 'classes': self.y[idx]}
        return sample

In [8]:
train_ds = CancerDataset(train[0],train[1])
test_ds = CancerDataset(test[0],test[1])

In [9]:
train_dataloader = DataLoader(train_ds, batch_size=20,shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_ds, batch_size=20,shuffle=True, num_workers=4)

In [10]:
def train_time(net, train_dataloader):
    criterion = nn.NLLLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    net.train()
    
    for epoch in range(100):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            # get the inputs
            inputs, labels = data['X'], data['classes']
            # wrap them in Variable
            inputs, labels = Variable(inputs.float()), Variable(labels)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.data[0]
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')
    return net

In [11]:
def test_time(net, test_dataloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.NLLLoss()
    for batch_idx, data in enumerate(test_dataloader):
        inputs, labels = data['X'], data['classes']
        inputs, targets = Variable(inputs.float()), Variable(labels)
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
    print "test set Accuracy", correct/float(total)

In [12]:
tnet = Net()
tnet = train_time(tnet,train_dataloader)


Finished Training


In [13]:
test_time(tnet,train_dataloader)

test set Accuracy 0.968


In [17]:
class BinarizeWeights(torch.autograd.Function):
    def __init__(self):
        super(BinarizeWeights, self).__init__()
    
    def forward(self, input, S):
        self.save_for_backward(S)
        res = torch.sign(input)
        return res

    def backward(self, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        S, = self.saved_tensors
        grad_input = torch.mm(grad_output, S)
        return grad_input

    

class BinaryLayer(nn.Linear):
    #initialize the Binary Layer where weights are binarized
    def __init__(self, input_dim, output_dim):
        super(BinaryLayer, self).__init__(input_dim, output_dim)
        
    def forward(self, x):
        self.new_weight = BinarizeWeights()(self.weight,x)
        # print self.new_weight.grad_fn
        backup_weight = self.weight.data
        self.weight.data = self.new_weight.data
        out = super(BinaryLayer, self).forward(x)
        return out

class Binaryactivation(torch.autograd.Function):
    #initialize the Binary Activation Function after Tanh
    def __init__(self):
        super(Binaryactivation, self).__init__()
        
    def forward(self, input):
        self.save_for_backward(input)
        out = torch.sign(input)
        return out
    
    def backward(self, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = self.saved_tensors
        grad_input = grad_output.clone()
        grad_input[torch.abs(input) >= 1] = 0
        return grad_input


    
class BinarytanH(torch.autograd.Function):
    #initialize the Binary Activation Function after Tanh
    def __init__(self):
        super(BinarytanH, self).__init__()
        
    def forward(self, x):
        res = F.tanh(x)
        out = Binaryactivation()(res)
#         print out.grad_fn
        return out


class BinaryNetwork(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(BinaryNetwork, self).__init__()
        self.linear1 = BinaryLayer(D_in, H)
        self.linear2 = BinaryLayer(H, D_out)
        self.binarized_tanh = BinarytanH()
        self.scores = nn.Softmax()
        
    def forward(self, x):
        linear_1 = self.linear1(x)
        activation_1 = self.binarized_tanh.forward(linear_1)
        smax_1 = self.linear2(activation_1)
        output = self.scores(smax_1)
        return output


In [18]:
binet = BinaryNetwork(9,50,2)
binet = train_time(binet,train_dataloader)

Finished Training


In [19]:
test_time(binet,train_dataloader)

test set Accuracy 0.43
